In [1]:
import torch
from torch import distributions as distro
from random import choices, sample, gauss
from itertools import combinations, permutations, combinations_with_replacement, product

In [ ]:
torch.

In [10]:
def sample_entity_subset(N_max, mu=3, sigma=0):
    N = min(N_max, round(gauss(mu, sigma)))
    entities = sample(list(range(N_max)), k=N)
    return entities

In [11]:
sample_entity_subset(6, 4, 0)

[4, 0, 1, 2]

In [12]:
def sample_interactions(N_max, N_entities, N_interactions):
    entity_subset = sample_entity_subset(N_max, mu=N_entities)
    comb = list(product(entity_subset, entity_subset))
    z = torch.zeros(N_max, N_max)
    num_interactions = min(round(gauss(mu=N_interactions, sigma=0)), len(entity_subset)**2)
    try:
        coord = sample(comb, k=num_interactions)
    except ValueError as e:
        print(e)
        print(f"k={num_interactions}, entity_subset={entity_subset}")
        return
    z[list(zip(*coord))] = 1.0
    return z, entity_subset

In [13]:
sample_interactions(6, 6, 8)

(tensor([[0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 1., 0.],
         [0., 0., 0., 0., 1., 0.],
         [1., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0.]]),
 [4, 5, 2, 3, 1, 0])

In [14]:
def sample_node_labels(N_entities, entity_subset, M_features=10):
    p = torch.full([N_entities, M_features], 0.5)
    u = torch.bernoulli(p)
    v = torch.zeros(N_entities, M_features)
    v[entity_subset] = u[entity_subset]
    return v

In [19]:
def interaction_samples(N_max, N_entities, N_interactions, M_features, iterations=10):
    interactions = []
    node_labels = []
    for i in range(iterations):
        z1, entity_subset = sample_interactions(N_max, N_entities, N_interactions)
        interactions.append(z1.unsqueeze(0))
        z2 = sample_node_labels(N_max, entity_subset, M_features)
        node_labels.append(z2.unsqueeze(0))
    interactions = torch.cat(interactions, 0)
    node_labels = torch.cat(node_labels, 0)
    return interactions, node_labels

In [20]:
z1 = interaction_samples(6, 4, 3, 10, 3)
z1

(tensor([[[0., 0., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0., 0.],
          [0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 1., 0., 0., 0.],
          [0., 0., 1., 0., 0., 0.]],
 
         [[1., 0., 0., 0., 1., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 1.],
          [0., 0., 0., 0., 0., 0.]]]),
 tensor([[[1., 0., 0., 1., 0., 1., 0., 0., 0., 0.],
          [1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 1., 0., 0., 1., 1., 1., 1., 1.],
          [1., 0., 1., 1., 1., 0., 0., 1., 1., 1.]],
 
         [[0., 0., 0., 0., 0

In [652]:
z2 = true_interaction_samples(6, 6, 12, 3)
z2

tensor([[[0., 1., 0., 0., 1., 1.],
         [0., 0., 0., 0., 0., 1.],
         [0., 0., 1., 1., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 1., 0.],
         [0., 0., 1., 1., 1., 0.]],

        [[0., 1., 0., 0., 0., 0.],
         [0., 1., 1., 1., 1., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 1., 1., 1., 0., 0.],
         [0., 1., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.]],

        [[0., 0., 1., 1., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [1., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1., 1.],
         [0., 1., 0., 0., 1., 0.],
         [1., 0., 0., 0., 1., 1.]]])

In [7]:
sample_node_labels(6, [0, 3, 4])

tensor([[1., 1., 1., 1., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 1.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [636]:
def true_node_label_samples(N_entities, entity_subset, M_features, iterations):
    samples = []
    for i in range(iterations):
        z = sample_node_labels(N_entities, entity_subset, M_features)
        samples.append(z.unsqueeze(0))
    samples = torch.cat(samples, 0)
    return samples

In [673]:
zz1 = true_node_label_samples(5, [0, 1, 4], 10, 3)
zz1

tensor([[[0., 0., 0., 1., 1., 0., 1., 0., 0., 1.],
         [0., 0., 0., 1., 0., 0., 0., 1., 1., 0.],
         [1., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
         [0., 1., 0., 0., 0., 1., 1., 0., 1., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 1., 0., 1., 0., 1., 0., 0.],
         [1., 1., 1., 0., 0., 1., 1., 0., 1., 0.],
         [0., 0., 0., 1., 0., 1., 0., 0., 0., 1.],
         [1., 1., 1., 1., 1., 0., 0., 0., 1., 1.],
         [0., 0., 0., 0., 1., 0., 1., 1., 1., 1.]],

        [[0., 0., 0., 0., 0., 1., 1., 1., 1., 0.],
         [0., 1., 0., 0., 1., 1., 0., 1., 0., 0.],
         [1., 1., 0., 1., 1., 1., 1., 0., 0., 0.],
         [1., 1., 0., 0., 1., 1., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0., 1., 0., 1., 0., 1.]]])

In [674]:
zz2 = true_node_label_samples(5, [1, 2, 3, 4], 10, 3)
zz2

tensor([[[1., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 1., 1., 0., 1., 0.],
         [1., 0., 1., 1., 0., 0., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1., 0., 0., 1., 0., 1.],
         [1., 1., 1., 0., 0., 1., 0., 1., 0., 1.]],

        [[0., 0., 0., 0., 1., 0., 0., 1., 1., 1.],
         [0., 0., 0., 1., 0., 0., 0., 1., 1., 1.],
         [0., 1., 1., 0., 0., 1., 1., 0., 0., 0.],
         [1., 1., 0., 1., 1., 1., 0., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 1., 1., 0., 1.]],

        [[0., 1., 1., 1., 1., 1., 0., 1., 0., 1.],
         [0., 1., 1., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 1., 1., 0., 1., 0., 0., 1., 0.],
         [0., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
         [1., 1., 0., 1., 0., 1., 0., 1., 1., 0.]]])

In [675]:
# https://github.com/napsternxg/pytorch-practice/blob/master/Pytorch%20-%20MMD%20VAE.ipynb
def compute_kernel(x, y):
    x_size = x.size(0)
    y_size = y.size(0)
    dim = x.size(1)
    x = x.unsqueeze(1) # (x_size, 1, dim)
    y = y.unsqueeze(0) # (1, y_size, dim)
    tiled_x = x.expand(x_size, y_size, dim)
    tiled_y = y.expand(x_size, y_size, dim)
    kernel_input = (tiled_x - tiled_y).pow(2).mean(2)/float(dim)
    return torch.exp(-kernel_input) # (x_size, y_size)

def mmd(x, y):
    x_kernel = compute_kernel(x, x)
    y_kernel = compute_kernel(y, y)
    xy_kernel = compute_kernel(x, y)
    mmd = x_kernel.mean() + y_kernel.mean() - 2*xy_kernel.mean()
    return mmd

In [676]:
mmd(z1.view(3, -1), z2.view(3, -1))

tensor(0.0073)

In [679]:
mmd(zz1.view(3, -1), zz2.view(3, -1))

tensor(0.0043)